## Initial setup

In [ ]:

!pip uninstall -y numpy
!pip install numpy==1.23.5

# numpy version
import os
os.kill(os.getpid(), 9)


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 68.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xgboost 2.1.1 requires nvidia-nccl-cu12; platform_system == "Linux" and platform_machine != "aarch64", which is not installed.
albucore 0.0.13 requires numpy<2,>=1.24.4, but you have numpy 1.23.5 which is incompatible.
albumentations 1.4.12 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
pandas-stubs 2.1.4.231227 requires numpy>=1.26.0; python_version < "3.13", but you have numpy 1.23.5 which is incompatible.


In [1]:
import numpy
numpy.__version__

'1.23.5'

In [2]:
import tensorflow as tf
print(tf.__version__)

2.17.0


In [3]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tqdm.notebook import tqdm
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
#import losses
import cv2
from imutils import paths
tf.random.set_seed(666)
np.random.seed(666)

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
import pandas as pd

In [6]:
import os

In [7]:
os.chdir("/content/drive/MyDrive/Colab Notebooks/research/Photocatalysis")

In [8]:
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks/research/Photocatalysis'

## Read the data

In [9]:
data = pd.read_csv('Photocatalysis_data.csv')

In [10]:
data

,Substrate species,Photocatalyst species,Selected wavelength (nm),Concentration (M),Photocatalyst ratio (mol%),Photocatalyst concentration (M),Reaction flow rate (μL/min),Laser light intensity (mW),non_stable,stable
0,S-1,Ir-1,293.3,0.005,0.5,0.000025,600,772,0.1173,0.0341
1,S-1,Ir-1,293.3,0.005,0.5,0.000025,600,1459,0.1600,0.0857
2,S-1,Ir-1,293.3,0.005,0.5,0.000025,600,2297,0.2091,0.1558
3,S-1,Ir-1,293.3,0.005,0.5,0.000025,600,3082,0.2204,0.2283
4,S-1,Ir-1,293.3,0.005,0.5,0.000025,600,3868,0.2890,0.2900
...,...,...,...,...,...,...,...,...,...,...
11995,S-5,Ir-5,293.3,0.020,2.0,0.000400,200,1459,0.2485,0.2668
11996,S-5,Ir-5,293.3,0.020,2.0,0.000400,200,2297,0.4914,0.4493
11997,S-5,Ir-5,293.3,0.020,2.0,0.000400,200,3082,0.6098,0.6022
11998,S-5,Ir-5,293.3,0.020,2.0,0.000400,200,3868,0.7205,0.6799


In [11]:
data['s_type'] = 100
data['lr_type'] = 100

for i in range(len(data)):
  data.at[i,'s_type'] = data.at[i,'Substrate species'][-1]
  data.at[i,'lr_type'] = data.at[i,'Photocatalyst species'][-1]

<ipython-input-11-d69fdb5ffd8c>:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.at[i,'s_type'] = data.at[i,'Substrate species'][-1]
<ipython-input-11-d69fdb5ffd8c>:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.at[i,'lr_type'] = data.at[i,'Photocatalyst species'][-1]


In [12]:
data.replace({'S-1':'C1(/C=C/COC/C=C/C2=CC=CC=C2)=CC=CC=C1','S-2':'C/C(C)=C/COC/C=C/C1=CC=CO1','S-3':'C/C(C)=C/COC/C=C(C)/C1=CC=CC=C1','S-4':'C/C(C)=C/COC/C=C/C1=CC=C(Cl)C=C1','S-5':'C/C(C)=C/COC/C=C/C1=CC=C(OC)C=C1'},inplace=True)

In [13]:
data.replace({'Ir-1':'FC(F)(C1=CN=C(C=C1)C2=C(F)C=C(F)C=[C-]2)F.F[P-](F)(F)(F)(F)F.CC(C)(C3=CC(C4=CC(C(C)(C)C)=CC=N4)=NC=C3)C.FC5=CC(F)=C([C-]=C5)C6=NC=C(C=C6)C(F)(F)F.[Ir+3]'
,'Ir-2':'CC1=CN=C(C2=C(C=C(C=[C-]2)F)F)C=C1.F[P-](F)(F)(F)(F)F.CC(C3=CC(C4=CC(C(C)(C)C)=CC=N4)=NC=C3)(C)C.FC5=CC(F)=C(C6=NC=C(C)C=C6)[C-]=C5.[Ir+3]'
,'Ir-3':'FC(C1=CN=C(C2=C(C=C(C=[C-]2)F)F)C=C1)(F)F.F[P-](F)(F)(F)(F)F.FC3=CC(F)=C(C4=NC=C(C(F)(F)F)C=C4)[C-]=C3.C5(C6=CC=CC=N6)=NC=CC=C5.[Ir+3]'
,'Ir-4':'CC1=CN=C(C2=C(C=C(C=[C-]2)F)F)C=C1.F[P-](F)(F)(F)(F)F.FC3=CC(F)=C(C4=NC=C(C)C=C4)[C-]=C3.C5(C6=CC=CC=N6)=NC=CC=C5.[Ir+3]'
,'Ir-5':'C1(C2=CC=CC=[C-]2)=NC=CC=C1.C3(C4=CC=CC=N4)=[C-]C=CC=C3.C5(C6=NC=CC=C6)=CC=CC=[C-]5.[Ir+3]'},inplace=True)

## Mordred fingerprint

In [14]:
!pip install mordred

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.0 MB/s eta 0:00:00
  Created wheel for mordred: filename=mordred-1.2.0-py3-none-any.whl size=176719 sha256=afa07125fef9a7bd373fa16639396234f1ec4e11bf8371357eab99fa1fbfafee
  Stored in directory: /root/.cache/pip/wheels/a7/4f/b8/d4c6591f6ac944aaced7865b349477695f662388ad958743c7
Successfully built mordred
  Attempting uninstall: networkx
    Found existing installation: networkx 3.3
    Uninstalling networkx-3.3:
      Successfully uninstalled networkx-3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cup

In [15]:
pip install rdkit==2023.3.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 35.4 MB/s eta 0:00:00


In [16]:
#!pip install --pre deepchem==2.7.2.dev20231011182505
!pip install rdkit-pypi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 44.1 MB/s eta 0:00:00


In [17]:
!pip install --pre deepchem==2.7.2.dev20231011182505

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.7/879.7 kB 20.6 MB/s eta 0:00:00


In [18]:
import pandas as pd
import numpy as np
import mordred
from rdkit import Chem
from rdkit.Chem import AllChem
from mordred import Calculator, descriptors

In [19]:
def mordred(data,name):
  df = data[[name]]
  # make rdkit mols
  df['rdmol'] = df[name].map(lambda x: Chem.MolFromSmiles(x))
  # drop molecules that rdkit cannot handle
  df= df.dropna(subset=['rdmol'])
  # get inchi
  df['inchi'] = df['rdmol'].map(Chem.MolToInchi)
  # drop duplicates on inchi
  df = df.drop_duplicates(subset="inchi")

  df['rdmolH'] = df['rdmol'].map(Chem.AddHs)
  _ = df['rdmolH'].map(AllChem.EmbedMolecule)
  _ = df['rdmolH'].map(AllChem.MMFFOptimizeMolecule)

  # initialize Mordred calculators
  calc=Calculator(descriptors, ignore_3D=True,)
  # compute Mordred descriptors for all molecules (may take long)
  md=calc.pandas(df['rdmol'])

  return pd.concat([df[[name]],md],axis=1)

In [20]:
names = ['Substrate species','Photocatalyst species']
prefix = ['substrate_','photocatalyst_']
#columns = ['a','b','c','d','e','f','g']
data1 = data[names]

for i in range(len(names)):
  tmp = mordred(data,names[i])
  tmp.columns = prefix[i] + tmp.columns
  data1 = pd.merge(data1,tmp,left_on=names[i],right_on=prefix[i]+names[i],how='left')
  del data1[names[i]]

<ipython-input-19-506492ad131d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rdmol'] = df[name].map(lambda x: Chem.MolFromSmiles(x))
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|██████████| 5/5 [00:00<00:00, 10.61it/s]
<ipython-input-19-506492ad131d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rdmol'] 

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [21]:
data2 = data1.select_dtypes(['number'])

### TSNE dimension reduction

In [22]:
from sklearn.manifold import TSNE

In [23]:
data_copy = pd.concat([data2.copy(deep=True),data.iloc[:,10:12],data.iloc[:,0:2]],axis=1).drop_duplicates(keep='first')
feature = data_copy.iloc[:,0:1433]
print(len(feature.drop_duplicates()))
feature = feature.values

X = feature
tsne = TSNE(n_components=2,perplexity=18,method='exact',random_state=0)
X_tsne = tsne.fit_transform(X)
#print(tsne.embedding_)

x_min, x_max = X_tsne.min(0), X_tsne.max(0)
X_norm = (X_tsne - x_min) / (x_max - x_min)  # 归一化

tsne = pd.DataFrame(X_norm)

5


In [24]:
tmp = data_copy[['Substrate species','Photocatalyst species']].reset_index()
del tmp['index']

In [25]:
tsne['Substrate species'] = tmp['Substrate species']
tsne['Photocatalyst species'] = tmp['Photocatalyst species']

In [26]:
tsne

,0,1,Substrate species,Photocatalyst species
0,0.000000,0.026131,C1(/C=C/COC/C=C/C2=CC=CC=C2)=CC=CC=C1,FC(F)(C1=CN=C(C=C1)C2=C(F)C=C(F)C=[C-]2)F.F[P-...
1,0.000000,0.026131,C1(/C=C/COC/C=C/C2=CC=CC=C2)=CC=CC=C1,CC1=CN=C(C2=C(C=C(C=[C-]2)F)F)C=C1.F[P-](F)(F)...
2,0.000000,0.026131,C1(/C=C/COC/C=C/C2=CC=CC=C2)=CC=CC=C1,FC(C1=CN=C(C2=C(C=C(C=[C-]2)F)F)C=C1)(F)F.F[P-...
3,0.000000,0.026131,C1(/C=C/COC/C=C/C2=CC=CC=C2)=CC=CC=C1,CC1=CN=C(C2=C(C=C(C=[C-]2)F)F)C=C1.F[P-](F)(F)...
4,0.000000,0.026131,C1(/C=C/COC/C=C/C2=CC=CC=C2)=CC=CC=C1,C1(C2=CC=CC=[C-]2)=NC=CC=C1.C3(C4=CC=CC=N4)=[C...
5,1.000000,0.000000,C/C(C)=C/COC/C=C/C1=CC=CO1,FC(F)(C1=CN=C(C=C1)C2=C(F)C=C(F)C=[C-]2)F.F[P-...
6,1.000000,0.000000,C/C(C)=C/COC/C=C/C1=CC=CO1,CC1=CN=C(C2=C(C=C(C=[C-]2)F)F)C=C1.F[P-](F)(F)...
7,1.000000,0.000000,C/C(C)=C/COC/C=C/C1=CC=CO1,FC(C1=CN=C(C2=C(C=C(C=[C-]2)F)F)C=C1)(F)F.F[P-...
8,1.000000,0.000000,C/C(C)=C/COC/C=C/C1=CC=CO1,CC1=CN=C(C2=C(C=C(C=[C-]2)F)F)C=C1.F[P-](F)(F)...
9,1.000000,0.000000,C/C(C)=C/COC/C=C/C1=CC=CO1,C1(C2=CC=CC=[C-]2)=NC=CC=C1.C3(C4=CC=CC=N4)=[C...


In [27]:
tmp1 = tsne.copy()

In [28]:
data_copy = pd.concat([data2.copy(deep=True),data.iloc[:,10:12],data.iloc[:,0:2]],axis=1).drop_duplicates(keep='first')
feature = data_copy.iloc[:,1433:2508]
print(len(feature.drop_duplicates()))
feature = feature.values

X = feature
tsne = TSNE(n_components=2,perplexity=18,method='exact',random_state=0)
X_tsne = tsne.fit_transform(X)
#print(tsne.embedding_)

x_min, x_max = X_tsne.min(0), X_tsne.max(0)
X_norm = (X_tsne - x_min) / (x_max - x_min)

tsne = pd.DataFrame(X_norm)

5


In [29]:
tsne_final = pd.concat([tsne,tmp1],axis=1)

In [30]:
tsne_final

,0,1,0,1,Substrate species,Photocatalyst species
0,0.000000,0.907579,0.000000,0.026131,C1(/C=C/COC/C=C/C2=CC=CC=C2)=CC=CC=C1,FC(F)(C1=CN=C(C=C1)C2=C(F)C=C(F)C=[C-]2)F.F[P-...
1,0.268066,1.000000,0.000000,0.026131,C1(/C=C/COC/C=C/C2=CC=CC=C2)=CC=CC=C1,CC1=CN=C(C2=C(C=C(C=[C-]2)F)F)C=C1.F[P-](F)(F)...
2,0.373904,0.429326,0.000000,0.026131,C1(/C=C/COC/C=C/C2=CC=CC=C2)=CC=CC=C1,FC(C1=CN=C(C2=C(C=C(C=[C-]2)F)F)C=C1)(F)F.F[P-...
3,0.624595,0.363287,0.000000,0.026131,C1(/C=C/COC/C=C/C2=CC=CC=C2)=CC=CC=C1,CC1=CN=C(C2=C(C=C(C=[C-]2)F)F)C=C1.F[P-](F)(F)...
4,1.000000,0.000000,0.000000,0.026131,C1(/C=C/COC/C=C/C2=CC=CC=C2)=CC=CC=C1,C1(C2=CC=CC=[C-]2)=NC=CC=C1.C3(C4=CC=CC=N4)=[C...
5,0.000000,0.907579,1.000000,0.000000,C/C(C)=C/COC/C=C/C1=CC=CO1,FC(F)(C1=CN=C(C=C1)C2=C(F)C=C(F)C=[C-]2)F.F[P-...
6,0.268066,1.000000,1.000000,0.000000,C/C(C)=C/COC/C=C/C1=CC=CO1,CC1=CN=C(C2=C(C=C(C=[C-]2)F)F)C=C1.F[P-](F)(F)...
7,0.373904,0.429326,1.000000,0.000000,C/C(C)=C/COC/C=C/C1=CC=CO1,FC(C1=CN=C(C2=C(C=C(C=[C-]2)F)F)C=C1)(F)F.F[P-...
8,0.624595,0.363287,1.000000,0.000000,C/C(C)=C/COC/C=C/C1=CC=CO1,CC1=CN=C(C2=C(C=C(C=[C-]2)F)F)C=C1.F[P-](F)(F)...
9,1.000000,0.000000,1.000000,0.000000,C/C(C)=C/COC/C=C/C1=CC=CO1,C1(C2=CC=CC=[C-]2)=NC=CC=C1.C3(C4=CC=CC=N4)=[C...


In [31]:
data3 = pd.merge(tsne_final,data,on=['Substrate species','Photocatalyst species'])

In [32]:
data_all_copy = data3.copy(deep=True)

In [33]:
del data3['Substrate species']
del data3['Photocatalyst species']

In [34]:
data3

,0,1,0,1,Selected wavelength (nm),Concentration (M),Photocatalyst ratio (mol%),Photocatalyst concentration (M),Reaction flow rate (μL/min),Laser light intensity (mW),non_stable,stable,s_type,lr_type
0,0.0,0.907579,0.000000,0.026131,293.3,0.005,0.5,0.000025,600,772,0.1173,0.0341,1,1
1,0.0,0.907579,0.000000,0.026131,293.3,0.005,0.5,0.000025,600,1459,0.1600,0.0857,1,1
2,0.0,0.907579,0.000000,0.026131,293.3,0.005,0.5,0.000025,600,2297,0.2091,0.1558,1,1
3,0.0,0.907579,0.000000,0.026131,293.3,0.005,0.5,0.000025,600,3082,0.2204,0.2283,1,1
4,0.0,0.907579,0.000000,0.026131,293.3,0.005,0.5,0.000025,600,3868,0.2890,0.2900,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,1.0,0.000000,0.257762,0.566530,293.3,0.020,2.0,0.000400,200,1459,0.2485,0.2668,5,5
11996,1.0,0.000000,0.257762,0.566530,293.3,0.020,2.0,0.000400,200,2297,0.4914,0.4493,5,5
11997,1.0,0.000000,0.257762,0.566530,293.3,0.020,2.0,0.000400,200,3082,0.6098,0.6022,5,5
11998,1.0,0.000000,0.257762,0.566530,293.3,0.020,2.0,0.000400,200,3868,0.7205,0.6799,5,5


In [35]:
for i in range(len(data3)):
  if data3.at[i,'stable'] < 0:
    data3.at[i,'stable'] = 0
  else:
    data3.at[i,'stable'] = float(data3.at[i,'stable'])

In [36]:
for i in range(len(data3)):
  if data3.at[i,'non_stable'] < 0:
    data3.at[i,'non_stable'] = 0
  else:
    data3.at[i,'non_stable'] = float(data3.at[i,'non_stable'])

In [37]:
data3['stable'] = data3['stable'].astype('float64')

In [38]:
data3['s_type'] = data3['s_type'].astype('float64')
data3['lr_type'] = data3['lr_type'].astype('float64')

In [39]:
data3['non_stable'] = data3['non_stable'].astype('float64')

In [40]:
data3.columns = [            'lr_0',             'lr_1',             's_0',             's_1',
         'Selected wavelength (nm)',     'Concentration (M)', 'Photocatalyst ratio (mol%)',    'Photocatalyst concentration (M)',
        'Reaction flow rate (μL/min)',      'Laser light intensity (mW)',  'non_stable',      'stable',
            's_type',     'lr_type']

In [41]:
data3

,lr_0,lr_1,s_0,s_1,Selected wavelength (nm),Concentration (M),Photocatalyst ratio (mol%),Photocatalyst concentration (M),Reaction flow rate (μL/min),Laser light intensity (mW),non_stable,stable,s_type,lr_type
0,0.0,0.907579,0.000000,0.026131,293.3,0.005,0.5,0.000025,600,772,0.1173,0.0341,1.0,1.0
1,0.0,0.907579,0.000000,0.026131,293.3,0.005,0.5,0.000025,600,1459,0.1600,0.0857,1.0,1.0
2,0.0,0.907579,0.000000,0.026131,293.3,0.005,0.5,0.000025,600,2297,0.2091,0.1558,1.0,1.0
3,0.0,0.907579,0.000000,0.026131,293.3,0.005,0.5,0.000025,600,3082,0.2204,0.2283,1.0,1.0
4,0.0,0.907579,0.000000,0.026131,293.3,0.005,0.5,0.000025,600,3868,0.2890,0.2900,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,1.0,0.000000,0.257762,0.566530,293.3,0.020,2.0,0.000400,200,1459,0.2485,0.2668,5.0,5.0
11996,1.0,0.000000,0.257762,0.566530,293.3,0.020,2.0,0.000400,200,2297,0.4914,0.4493,5.0,5.0
11997,1.0,0.000000,0.257762,0.566530,293.3,0.020,2.0,0.000400,200,3082,0.6098,0.6022,5.0,5.0
11998,1.0,0.000000,0.257762,0.566530,293.3,0.020,2.0,0.000400,200,3868,0.7205,0.6799,5.0,5.0


In [42]:
data3.to_csv('data_after_preprocessing_0807.csv',index=False)